<a href="https://colab.research.google.com/github/unesprodrigofct/BIgData/blob/master/SystemRecommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import datetime
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
#Load movielens data

genome_scores_data = pd.read_csv('genome-scores.csv') 
movies_data = pd.read_csv('movies.csv') 
movies_data2 = pd.read_csv('movies.csv') 
ratings_data = pd.read_csv('ratings.csv')

FileNotFoundError: ignored

In [0]:
genome_scores_data.head()

,movieId,tagId,relevance
0,1,1,0.02500
1,1,2,0.02500
2,1,3,0.05775
3,1,4,0.09675
4,1,5,0.14675


In [0]:
movies_data.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [0]:
ratings_data.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


##  ***Prepare data for movies matrix***

We create 3 dfs (mov_tag_df, mov_genres_df, mov_rating_df) and calculate 3 cosine similarities for each of them. After that, we mix them and we obtain the movies matrix. While we mix the datasets, we use a formula (mov_tag_df*0.5+mov_genres_df*0.25+mov_rating_df*0.25). In this case tags are the most important data to calculate similarity so it should effect to similarity calculation more than others. Ok, let’s start to create dfs

mov_tag_df

mov_tag_df is created with using “genome_scores.csv”. We pivot the data to compare movies through the tags.


In [0]:
scores_pivot = genome_scores_data.pivot_table(index = ["movieId"],columns = ["tagId"],values = "relevance").reset_index()
scores_pivot.head()

tagId,movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1089,1090,1091,1092,1093,1094,1095,1096,1097,1098,1099,1100,1101,1102,1103,1104,1105,1106,1107,1108,1109,1110,1111,1112,1113,1114,1115,1116,1117,1118,1119,1120,1121,1122,1123,1124,1125,1126,1127,1128
0,1,0.02500,0.02500,0.05775,0.09675,0.14675,0.21700,0.06700,0.26275,0.26200,0.03200,0.57700,0.11625,0.18800,0.00800,0.03675,0.28175,0.00700,0.11050,0.67050,0.18450,0.33025,0.28250,0.05700,0.01550,0.08500,0.08100,0.19500,0.07150,0.89200,0.67625,0.03875,0.22800,0.40200,0.03875,0.02675,0.33025,0.10100,0.01250,0.01850,...,0.10850,0.60425,0.41050,0.44500,0.33725,0.02175,0.04075,0.06250,0.04375,0.10075,0.00475,0.19225,0.25850,0.00900,0.02775,0.29925,0.04675,0.01025,0.02725,0.73700,0.11200,0.05125,0.04375,0.05350,0.12575,0.77675,0.14500,0.11275,0.04200,0.10250,0.03950,0.01800,0.04575,0.03275,0.12500,0.04150,0.01925,0.03625,0.07775,0.02300
1,2,0.03975,0.04375,0.03775,0.04800,0.11025,0.07250,0.04775,0.10975,0.09925,0.02050,0.06775,0.08900,0.22575,0.00625,0.00300,0.03475,0.00950,0.18975,0.64600,0.40025,0.45100,0.60200,0.15100,0.28100,0.07600,0.14350,0.11675,0.04350,0.98100,0.10550,0.00825,0.06650,0.08575,0.05300,0.04525,0.14650,0.07750,0.02900,0.02275,...,0.08925,0.32475,0.19125,0.32550,0.17675,0.02650,0.03500,0.04125,0.02775,0.04750,0.00575,0.04700,0.15700,0.01425,0.02400,0.31600,0.08850,0.02375,0.01300,0.24450,0.10075,0.03425,0.02475,0.26450,0.39025,0.18000,0.18725,0.14750,0.01500,0.05700,0.04175,0.01925,0.01725,0.02425,0.12550,0.02250,0.01550,0.01475,0.09025,0.01875
2,3,0.04350,0.05475,0.02800,0.07700,0.05400,0.06850,0.05600,0.18500,0.04925,0.02675,0.02225,0.07100,0.09050,0.00475,0.00250,0.02875,0.01175,0.10400,0.16475,0.17375,0.28000,0.20775,0.08675,0.01700,0.04250,0.08375,0.12225,0.10800,0.41200,0.09525,0.00900,0.04750,0.06475,0.05925,0.02600,0.20950,0.04375,0.02350,0.05675,...,0.10525,0.11400,0.21250,0.16575,0.03550,0.02450,0.04925,0.03825,0.03400,0.07225,0.01125,0.03875,0.13875,0.41600,0.04175,0.16800,0.06175,0.02050,0.04275,0.15525,0.17250,0.04275,0.04450,0.02325,0.03375,0.19950,0.02825,0.37075,0.02625,0.07325,0.04150,0.02675,0.02775,0.03425,0.15550,0.03675,0.01700,0.01950,0.09700,0.01850
3,4,0.03725,0.03950,0.03675,0.03100,0.06825,0.04050,0.02325,0.08700,0.05125,0.03025,0.02125,0.03325,0.11750,0.01075,0.00275,0.02875,0.01400,0.16875,0.13950,0.12900,0.49875,0.44650,0.09050,0.01525,0.06175,0.06175,0.08850,0.55575,0.13800,0.10575,0.01050,0.03925,0.07950,0.05300,0.01725,0.05650,0.09025,0.01275,0.01125,...,0.09350,0.12250,0.18575,0.18000,0.04800,0.01625,0.01675,0.03625,0.02050,0.05800,0.01700,0.03025,0.31475,0.09275,0.02825,0.17850,0.06750,0.02875,0.02975,0.16875,0.06225,0.16425,0.03925,0.01875,0.03400,0.10725,0.02825,0.97675,0.01800,0.28825,0.05750,0.03375,0.02275,0.03975,0.18525,0.05925,0.01500,0.01525,0.06450,0.01300
4,5,0.04200,0.05275,0.05925,0.03675,0.07525,0.12525,0.02850,0.08500,0.02950,0.02875,0.03125,0.06150,0.07175,0.00650,0.00225,0.02775,0.01425,0.16100,0.16350,0.22650,0.27575,0.21075,0.10275,0.02250,0.04150,0.05100,0.19200,0.17150,0.15375,0.12225,0.01000,0.04175,0.07950,0.04050,0.03175,0.07250,0.03400,0.01925,0.04200,...,0.10825,0.14700,0.18750,0.22100,0.08950,0.01350,0.04075,0.03550,0.03075,0.06625,0.00775,0.03850,0.13150,0.80675,0.02650,0.26975,0.05575,0.02100,0.02175,0.20825,0.11850,0.14150,0.03875,0.03475,0.04675,0.12000,0.02925,0.48900,0.02150,0.07450,0.04250,0.02825,0.02150,0.02600,0.14275,0.02075,0.01650,0.01675,0.10750,0.01825


We need to join “scores_pivot” df with “movies_data” df to get all movieIds. Then we fill null values and drop columns which are not used


In [0]:
mov_tag_df = movies_data.merge(scores_pivot, left_on='movieId', right_on='movieId', how='left')
mov_tag_df = mov_tag_df.fillna(0) 
mov_tag_df = mov_tag_df.drop(['title','genres'], axis = 1)
mov_tag_df.head()

,movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1089,1090,1091,1092,1093,1094,1095,1096,1097,1098,1099,1100,1101,1102,1103,1104,1105,1106,1107,1108,1109,1110,1111,1112,1113,1114,1115,1116,1117,1118,1119,1120,1121,1122,1123,1124,1125,1126,1127,1128
0,1,0.02500,0.02500,0.05775,0.09675,0.14675,0.21700,0.06700,0.26275,0.26200,0.03200,0.57700,0.11625,0.18800,0.00800,0.03675,0.28175,0.00700,0.11050,0.67050,0.18450,0.33025,0.28250,0.05700,0.01550,0.08500,0.08100,0.19500,0.07150,0.89200,0.67625,0.03875,0.22800,0.40200,0.03875,0.02675,0.33025,0.10100,0.01250,0.01850,...,0.10850,0.60425,0.41050,0.44500,0.33725,0.02175,0.04075,0.06250,0.04375,0.10075,0.00475,0.19225,0.25850,0.00900,0.02775,0.29925,0.04675,0.01025,0.02725,0.73700,0.11200,0.05125,0.04375,0.05350,0.12575,0.77675,0.14500,0.11275,0.04200,0.10250,0.03950,0.01800,0.04575,0.03275,0.12500,0.04150,0.01925,0.03625,0.07775,0.02300
1,2,0.03975,0.04375,0.03775,0.04800,0.11025,0.07250,0.04775,0.10975,0.09925,0.02050,0.06775,0.08900,0.22575,0.00625,0.00300,0.03475,0.00950,0.18975,0.64600,0.40025,0.45100,0.60200,0.15100,0.28100,0.07600,0.14350,0.11675,0.04350,0.98100,0.10550,0.00825,0.06650,0.08575,0.05300,0.04525,0.14650,0.07750,0.02900,0.02275,...,0.08925,0.32475,0.19125,0.32550,0.17675,0.02650,0.03500,0.04125,0.02775,0.04750,0.00575,0.04700,0.15700,0.01425,0.02400,0.31600,0.08850,0.02375,0.01300,0.24450,0.10075,0.03425,0.02475,0.26450,0.39025,0.18000,0.18725,0.14750,0.01500,0.05700,0.04175,0.01925,0.01725,0.02425,0.12550,0.02250,0.01550,0.01475,0.09025,0.01875
2,3,0.04350,0.05475,0.02800,0.07700,0.05400,0.06850,0.05600,0.18500,0.04925,0.02675,0.02225,0.07100,0.09050,0.00475,0.00250,0.02875,0.01175,0.10400,0.16475,0.17375,0.28000,0.20775,0.08675,0.01700,0.04250,0.08375,0.12225,0.10800,0.41200,0.09525,0.00900,0.04750,0.06475,0.05925,0.02600,0.20950,0.04375,0.02350,0.05675,...,0.10525,0.11400,0.21250,0.16575,0.03550,0.02450,0.04925,0.03825,0.03400,0.07225,0.01125,0.03875,0.13875,0.41600,0.04175,0.16800,0.06175,0.02050,0.04275,0.15525,0.17250,0.04275,0.04450,0.02325,0.03375,0.19950,0.02825,0.37075,0.02625,0.07325,0.04150,0.02675,0.02775,0.03425,0.15550,0.03675,0.01700,0.01950,0.09700,0.01850
3,4,0.03725,0.03950,0.03675,0.03100,0.06825,0.04050,0.02325,0.08700,0.05125,0.03025,0.02125,0.03325,0.11750,0.01075,0.00275,0.02875,0.01400,0.16875,0.13950,0.12900,0.49875,0.44650,0.09050,0.01525,0.06175,0.06175,0.08850,0.55575,0.13800,0.10575,0.01050,0.03925,0.07950,0.05300,0.01725,0.05650,0.09025,0.01275,0.01125,...,0.09350,0.12250,0.18575,0.18000,0.04800,0.01625,0.01675,0.03625,0.02050,0.05800,0.01700,0.03025,0.31475,0.09275,0.02825,0.17850,0.06750,0.02875,0.02975,0.16875,0.06225,0.16425,0.03925,0.01875,0.03400,0.10725,0.02825,0.97675,0.01800,0.28825,0.05750,0.03375,0.02275,0.03975,0.18525,0.05925,0.01500,0.01525,0.06450,0.01300
4,5,0.04200,0.05275,0.05925,0.03675,0.07525,0.12525,0.02850,0.08500,0.02950,0.02875,0.03125,0.06150,0.07175,0.00650,0.00225,0.02775,0.01425,0.16100,0.16350,0.22650,0.27575,0.21075,0.10275,0.02250,0.04150,0.05100,0.19200,0.17150,0.15375,0.12225,0.01000,0.04175,0.07950,0.04050,0.03175,0.07250,0.03400,0.01925,0.04200,...,0.10825,0.14700,0.18750,0.22100,0.08950,0.01350,0.04075,0.03550,0.03075,0.06625,0.00775,0.03850,0.13150,0.80675,0.02650,0.26975,0.05575,0.02100,0.02175,0.20825,0.11850,0.14150,0.03875,0.03475,0.04675,0.12000,0.02925,0.48900,0.02150,0.07450,0.04250,0.02825,0.02150,0.02600,0.14275,0.02075,0.01650,0.01675,0.10750,0.01825


####mov_genres_df
mov_genres_df is created with using “movies.csv”. We split genres field for each movies and then we create columns for each genres. We define a function to split genres column and check it if it exists or not

In [0]:
def set_genres(genres,col):
    if genres in col.split('|'): return 1
    else: return 0

In [0]:
mov_genres_df = movies_data
mov_genres_df["Action"] = mov_genres_df.apply(lambda x: set_genres("Action",x['genres']), axis=1)
mov_genres_df["Adventure"] = mov_genres_df.apply(lambda x: set_genres("Adventure",x['genres']), axis=1)
mov_genres_df["Animation"] = mov_genres_df.apply(lambda x: set_genres("Animation",x['genres']), axis=1)
mov_genres_df["Children"] = mov_genres_df.apply(lambda x: set_genres("Children",x['genres']), axis=1)
mov_genres_df["Comedy"] = mov_genres_df.apply(lambda x: set_genres("Comedy",x['genres']), axis=1)
mov_genres_df["Crime"] = mov_genres_df.apply(lambda x: set_genres("Crime",x['genres']), axis=1)
mov_genres_df["Documentary"] = mov_genres_df.apply(lambda x: set_genres("Documentary",x['genres']), axis=1)
mov_genres_df["Drama"] = mov_genres_df.apply(lambda x: set_genres("Drama",x['genres']), axis=1)
mov_genres_df["Fantasy"] = mov_genres_df.apply(lambda x: set_genres("Fantasy",x['genres']), axis=1)
mov_genres_df["Film-Noir"] = mov_genres_df.apply(lambda x: set_genres("Film-Noir",x['genres']), axis=1)
mov_genres_df["Horror"] = mov_genres_df.apply(lambda x: set_genres("Horror",x['genres']), axis=1)
mov_genres_df["Musical"] = mov_genres_df.apply(lambda x: set_genres("Musical",x['genres']), axis=1)
mov_genres_df["Mystery"] = mov_genres_df.apply(lambda x: set_genres("Mystery",x['genres']), axis=1)
mov_genres_df["Romance"] = mov_genres_df.apply(lambda x: set_genres("Romance",x['genres']), axis=1)
mov_genres_df["Sci-Fi"] = mov_genres_df.apply(lambda x: set_genres("Sci-Fi",x['genres']), axis=1)
mov_genres_df["Thriller"] = mov_genres_df.apply(lambda x: set_genres("Thriller",x['genres']), axis=1)
mov_genres_df["War"] = mov_genres_df.apply(lambda x: set_genres("War",x['genres']), axis=1)
mov_genres_df["Western"] = mov_genres_df.apply(lambda x: set_genres("Western",x['genres']), axis=1)
mov_genres_df["(no genres listed)"] = mov_genres_df.apply(lambda x: set_genres("(no genres listed)",x['genres']), axis=1)

In [0]:
mov_genres_df.drop(['title','genres'], axis = 1, inplace=True)
mov_genres_df.head()

,movieId,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,(no genres listed)
0,1,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,2,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,4,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0
4,5,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#####mov_rating_df
mov_rating_df is created with using “movies.csv” and “ratings.csv”. It includes year, rating and rating count information. We extract year information from “title” field. Then we group years through 0–5. We calculate mean of rating and counts of ratings for each movie. Then we group rating counts through 0–5. The reason why we group years and rating counts is that to reduce the scale and it helps to calculate better similarity. Firstly, we define a function to extract year information

In [0]:
def set_year(title):
    year = title.strip()[-5:-1]
    if str(year).isnumeric() == True: return int(year)
    else: return 1800

movies_data2['year'] = movies_data2.apply(lambda x: set_year(x['title']), axis=1)
movies = movies_data2.drop(['genres'], axis = 1)
movies.head()

,movieId,title,year
0,1,Toy Story (1995),1995
1,2,Jumanji (1995),1995
2,3,Grumpier Old Men (1995),1995
3,4,Waiting to Exhale (1995),1995
4,5,Father of the Bride Part II (1995),1995


In [0]:
#define function to group years
def set_year_group(year):
    if (year < 1900): return 0
    elif (1900 <= year <= 1975): return 1
    elif (1976 <= year <= 1995): return 2
    elif (1996 <= year <= 2003): return 3
    elif (2004 <= year <= 2009): return 4
    elif (2010 <= year): return 5
    else: return 0
movies['year_group'] = movies.apply(lambda x: set_year_group(x['year']), axis=1)
#no need title and year fields
movies.drop(['title','year'], axis = 1, inplace=True)
movies.head()

,movieId,year_group
0,1,2
1,2,2
2,3,2
3,4,2
4,5,2


In [0]:
agg_movies_rat = ratings_data.groupby(['movieId']).agg({'rating': [np.size, np.mean]}).reset_index()
agg_movies_rat.columns = ['movieId','rating_counts', 'rating_mean']
agg_movies_rat.head()

,movieId,rating_counts,rating_mean
0,1,4289.0,3.930520
1,2,1942.0,3.230947
2,3,1111.0,3.170567
3,4,251.0,2.968127
4,5,1088.0,3.117188


In [0]:
#define function to group rating counts
def set_rating_group(rating_counts):
    if (rating_counts <= 1): return 0
    elif (2 <= rating_counts <= 10): return 1
    elif (11 <= rating_counts <= 100): return 2
    elif (101 <= rating_counts <= 1000): return 3
    elif (1001 <= rating_counts <= 5000): return 4
    elif (5001 <= rating_counts): return 5
    else: return 0
agg_movies_rat['rating_group'] = agg_movies_rat.apply(lambda x: set_rating_group(x['rating_counts']), axis=1)
#no need rating_counts field
agg_movies_rat.drop('rating_counts', axis = 1, inplace=True)
mov_rating_df = movies.merge(agg_movies_rat, left_on='movieId', right_on='movieId', how='left')
mov_rating_df = mov_rating_df.fillna(0)
mov_rating_df.head()

,movieId,year_group,rating_mean,rating_group
0,1,2,3.930520,4.0
1,2,2,3.230947,4.0
2,3,2,3.170567,4.0
3,4,2,2.968127,3.0
4,5,2,3.117188,4.0


## **movies matrix**
We create 3 different datasets to calculate similarity. We use cosine similarity. Cosine similarity is a measure of similarity between two non-zero vectors of an inner product space that measures the cosine of the angle between them. I use cosine similarity because I think it often allows better meaning due to the high dimensionality. Before we calculate cosine similarity, we should set “movieId” field as index in the dfs

In [0]:
mov_tag_df = mov_tag_df.set_index('movieId')
mov_genres_df = mov_genres_df.set_index('movieId')
mov_rating_df = mov_rating_df.set_index('movieId')

In [0]:
#cosine similarity for mov_tag_df
cos_tag = cosine_similarity(mov_tag_df.values)*0.5
#cosine similarity for mov_genres_df
cos_genres = cosine_similarity(mov_genres_df.values)*0.25
#cosine similarity for mov_rating_df
cos_rating = cosine_similarity(mov_rating_df.values)*0.25
#mix
cos = cos_tag+cos_genres+cos_rating

In [0]:
cols = mov_tag_df.index.values
inx = mov_tag_df.index
movies_sim = pd.DataFrame(cos, columns=cols, index=inx)
movies_sim.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,131126,131128,131130,131132,131134,131136,131138,131140,131142,131144,131146,131148,131150,131152,131154,131156,131158,131160,131162,131164,131166,131168,131170,131172,131174,131176,131180,131231,131237,131239,131241,131243,131248,131250,131252,131254,131256,131258,131260,131262
movieId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.829694,0.680409,0.639815,0.695784,0.561469,0.653712,0.766346,0.518534,0.631623,0.650978,0.606735,0.829484,0.543079,0.610246,0.555790,0.554247,0.674238,0.631890,0.611321,0.640842,0.543385,0.524629,0.571748,0.547341,0.591342,0.660869,0.538970,0.662587,0.537182,0.581252,0.557576,0.638129,0.756397,0.530199,0.552924,0.598543,0.717357,0.637908,0.546703,...,0.195782,0.195782,0.242092,0.242092,0.195782,0.195782,0.148528,0.083978,0.195782,0.242092,0.242092,0.163035,0.195782,0.195782,0.195782,0.195782,0.195782,0.083978,0.083978,0.083978,0.083978,0.083978,0.083978,0.083978,0.083978,0.083978,0.083978,0.163035,0.195782,0.163035,0.163035,0.242092,0.333978,0.195782,0.163035,0.195782,0.195782,0.195782,0.083978,0.213078
2,0.829694,1.000000,0.593249,0.567614,0.589422,0.510825,0.568331,0.839132,0.569077,0.661614,0.566515,0.542646,0.750342,0.509341,0.689033,0.503576,0.514544,0.537031,0.529512,0.585699,0.535291,0.544619,0.540904,0.576184,0.496049,0.552156,0.670262,0.505818,0.652278,0.486568,0.600221,0.525468,0.658940,0.724066,0.498700,0.514485,0.578367,0.676652,0.538110,0.522470,...,0.090626,0.090626,0.090626,0.090626,0.090626,0.090626,0.090626,0.090626,0.090626,0.090626,0.090626,0.090626,0.090626,0.090626,0.090626,0.090626,0.090626,0.090626,0.090626,0.090626,0.090626,0.090626,0.090626,0.090626,0.090626,0.090626,0.090626,0.090626,0.090626,0.090626,0.090626,0.090626,0.284276,0.090626,0.090626,0.090626,0.090626,0.234964,0.090626,0.257293
3,0.680409,0.593249,1.000000,0.832571,0.857074,0.570058,0.888619,0.609254,0.558193,0.587187,0.809035,0.725910,0.550044,0.550916,0.685490,0.554532,0.681720,0.765811,0.799800,0.691493,0.687033,0.570173,0.561909,0.586584,0.683376,0.591958,0.598262,0.671284,0.532365,0.537384,0.591591,0.545997,0.697224,0.575602,0.683580,0.561410,0.588566,0.693437,0.827759,0.568711,...,0.267984,0.267984,0.216207,0.216207,0.267984,0.267984,0.193269,0.091207,0.267984,0.216207,0.216207,0.341207,0.267984,0.267984,0.267984,0.267984,0.267984,0.091207,0.216207,0.091207,0.091207,0.091207,0.091207,0.091207,0.193269,0.091207,0.091207,0.341207,0.267984,0.216207,0.341207,0.216207,0.170264,0.267984,0.216207,0.267984,0.267984,0.091207,0.091207,0.091207
4,0.639815,0.567614,0.832571,1.000000,0.766271,0.554474,0.863200,0.623578,0.540970,0.548802,0.885082,0.657136,0.552981,0.688781,0.642691,0.643734,0.816341,0.704948,0.665877,0.721532,0.639958,0.628235,0.542311,0.707311,0.784866,0.745164,0.739149,0.799955,0.590211,0.644865,0.760343,0.521223,0.676960,0.689277,0.812921,0.669571,0.593346,0.703834,0.803820,0.733061,...,0.251402,0.251402,0.209126,0.209126,0.251402,0.251402,0.273731,0.251402,0.251402,0.209126,0.209126,0.311188,0.251402,0.251402,0.251402,0.251402,0.251402,0.251402,0.311188,0.107064,0.107064,0.251402,0.107064,0.107064,0.273731,0.251402,0.107064,0.311188,0.251402,0.311188,0.311188,0.209126,0.171614,0.251402,0.209126,0.251402,0.251402,0.107064,0.107064,0.107064
5,0.695784,0.589422,0.857074,0.766271,1.000000,0.511492,0.817536,0.583355,0.537418,0.548195,0.750064,0.742204,0.540233,0.505014,0.551855,0.492327,0.541345,0.789439,0.829644,0.688530,0.673079,0.532499,0.520544,0.576082,0.513944,0.546077,0.580421,0.517205,0.496083,0.493446,0.574808,0.499760,0.547260,0.580530,0.519779,0.520490,0.556403,0.774307,0.734813,0.527795,...,0.341721,0.341721,0.268498,0.268498,0.341721,0.341721,0.236059,0.091721,0.341721,0.268498,0.268498,0.268498,0.341721,0.341721,0.341721,0.341721,0.341721,0.091721,0.091721,0.091721,0.091721,0.091721,0.091721,0.091721,0.091721,0.091721,0.091721,0.

In [0]:
def get_similar(movieId):
    df = movies_sim.loc[movies_sim.index == movieId].reset_index(). \
            melt(id_vars='movieId', var_name='sim_moveId', value_name='relevance'). \
            sort_values('relevance', axis=0, ascending=False)[1:6]
    return df
#create empty df
movies_similarity = pd.DataFrame(columns=['movieId','sim_moveId','relevance'])

In [0]:
for x in movies_sim.index.tolist():
    movies_similarity = movies_similarity.append(get_similar(x))
movies_similarity.head()

,movieId,sim_moveId,relevance
582,1,588,0.890041
1515,1,1566,0.868452
1931,1,2015,0.855987
1970,1,2054,0.850498
1765,1,1848,0.850038
...,...,...,...
22750,131262,108882,0.473607
10544,131262,39801,0.466506
23115,131262,110216,0.466506
20939,131262,102250,0.466506


### **Check recommendations**
Data is ready to import graph db. We already calculate similarity of movies so we can define a function to get 5 the most similar movies for a movie.

In [0]:
def movie_recommender(movieId):
    df = movies_sim.loc[movies_sim.index == movieId].reset_index(). \
            melt(id_vars='movieId', var_name='sim_moveId', value_name='relevance'). \
            sort_values('relevance', axis=0, ascending=False)[1:6]
    df['sim_moveId'] = df['sim_moveId'].astype(int)
    sim_df = movies_data.merge(df, left_on='movieId', right_on='sim_moveId', how='inner'). \
                sort_values('relevance', axis=0, ascending=False). \
                loc[: , ['movieId_y','title','genres']]. \
                rename(columns={ 'movieId_y': "movieId" })
    return sim_df

In [0]:
#get recommendation for Toy Story
movie_recommender(1)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,movieId,title,genres
0,1,NaN,NaN
1,1,NaN,NaN
3,1,NaN,NaN
4,1,NaN,NaN
2,1,NaN,NaN
